In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model , Sequential
import seaborn as sns 
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D ,Conv2D
from keras.layers import Activation , Dropout , Flatten , Dense 
from keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
import numpy as np
%matplotlib inline
from keras.models import load_model
import cv2
from keras.callbacks import ModelCheckpoint, EarlyStopping , ReduceLROnPlateau
import tensorflow as tf 
from keras.preprocessing import image 
from keras.preprocessing.image import img_to_array
from IPython.display import SVG, Image
from PIL import Image
from IPython import display
from tensorflow.keras.optimizers import Adam

In [7]:
img_size = 48
batch_size = 64
datagen_train = ImageDataGenerator(horizontal_flip = True)
datagen_validation = ImageDataGenerator(horizontal_flip = True)
train_generator = datagen_train.flow_from_directory('/Users/shreeyarao/Desktop/train' , target_size = (img_size , img_size) , color_mode = 'grayscale', batch_size = batch_size , class_mode = 'categorical' , shuffle = True)
validation_generator = datagen_train.flow_from_directory('/Users/shreeyarao/Desktop/test' , target_size = (img_size , img_size) , color_mode = 'grayscale', batch_size = batch_size , class_mode = 'categorical' , shuffle = True)


Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


CNN MODEL


In [3]:
model = Sequential()

model.add(Conv2D(64,(3,3), padding = 'same' , input_shape = (48,48,1)))

#we have added here a Conv2D nd passes in 64 filters each of 3X3 with same padding and a specific input shape of 48,48
# here 1 in 48,48,1 refers to the fact that the image is in grayscale 

#add batch normalization and activation function

model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

#2nd Conv Layer 

model.add(Conv2D(128,(5,5), padding = 'same' , input_shape = (48,48,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

#3rd Conv Layer 

model.add(Conv2D(128,(5,5), padding = 'same' , input_shape = (48,48,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

#4th Conv Layer

model.add(Conv2D(512,(3,3), padding = 'same' , input_shape = (48,48,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

#end of Conv layers

#flatten the output 
model.add(Flatten())

#addition of fully connected layers

model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))


model.add(Dense(7, activation= 'softmax'))
opt = Adam( learning_rate = 0.0005)
model.compile(optimizer = opt , loss = 'categorical_crossentropy' , metrics = ['accuracy'])
model.summary()




Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 64)        640       
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 48, 64)        256       
_________________________________________________________________
activation (Activation)      (None, 48, 48, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 128)       204928    
_________________________________________________________________
batch_normalization_1 (Batch (None, 24, 24, 128)       5

TRAINING AND EVALUATING THE MODEL

In [4]:
epochs = 15
steps_per_epoch = train_generator.n // train_generator.batch_size
validation_steps = validation_generator.n // validation_generator.batch_size
checkpoint = ModelCheckpoint('model_weights.h5' , monitor = 'val_accuracy' , save_weights_only = True , mode = 'max', verbose = 1)

early_stopping = EarlyStopping(   monitor = 'val_loss' , min_delta = 0 , patience = 3 , verbose = 1, restore_best_weights = True)

reduce_learningrate =  ReduceLROnPlateau( monitor = 'val_loss' , factor = 0.2 , patience = 3 , verbose = 1 , min_delta = 0.0001)



callbacks_list = [early_stopping , checkpoint ,  reduce_learningrate]





history = model.fit(
          x=train_generator,
                                steps_per_epoch=train_generator.n//train_generator.batch_size,
                                epochs=epochs,
                                validation_data = validation_generator,
                               validation_steps = validation_generator.n // validation_generator.batch_size , 
                                callbacks=callbacks_list
  
    )



model.save("model.h5")
model.load_weights('model.h5')




Epoch 1/15
448/448 [==============================] - 643s 1s/step - loss: 1.8136 - accuracy: 0.3013 - val_loss: 1.6924 - val_accuracy: 0.3788

Epoch 00001: saving model to model_weights.h5
Epoch 2/15
448/448 [==============================] - 677s 2s/step - loss: 1.5081 - accuracy: 0.4215 - val_loss: 1.3651 - val_accuracy: 0.4780

Epoch 00002: saving model to model_weights.h5
Epoch 3/15
448/448 [==============================] - 622s 1s/step - loss: 1.3577 - accuracy: 0.4809 - val_loss: 1.3138 - val_accuracy: 0.4773

Epoch 00003: saving model to model_weights.h5
Epoch 4/15
448/448 [==============================] - 684s 2s/step - loss: 1.2747 - accuracy: 0.5146 - val_loss: 1.3530 - val_accuracy: 0.4852

Epoch 00004: saving model to model_weights.h5
Epoch 5/15
448/448 [==============================] - 1163s 3s/step - loss: 1.2161 - accuracy: 0.5375 - val_loss: 1.4326 - val_accuracy: 0.4442

Epoch 00005: saving model to model_weights.h5
Epoch 6/15
448/448 [=============================


        open cv part
        

In [8]:

classifier  = load_model('model.h5')
img_path = '/Users/shreeyarao/Desktop/sad.jpg'
image = image.load_img(img_path)
prediction = classifier.predict(image)
print(prediction)

ValueError: Failed to find data adapter that can handle input: <class 'PIL.Image.Image'>, <class 'NoneType'>

In [4]:
classifier  = load_model('model.h5')
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
emotion_labels = ['Angry','Disgust','Fear' ,'Happy','Neutral','Sad','Surprise']

cap = cv2.VideoCapture(0) 

while True:
    
    _, frame = cap.read()
    labels = []
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray)
    
    
    
    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
        roi_gray = gray[y:y+h,x:x+w]
        roi_gray = cv2.resize(roi_gray,(48,48),interpolation=cv2.INTER_AREA)
        
        
        if np.sum([roi_gray])!=0:
            roi = roi_gray.astype('float')/255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi,axis=0)

            prediction = classifier.predict(roi)[0]
            
            label = emotion_labels[prediction.argmax()]
            print(label)
            label_position = (x,y-10)
            cv2.putText(frame,label,label_position,cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
        else:
            cv2.putText(frame,'No Faces',(30,80),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
    cv2.imshow('Emotion Detector',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
        
        
        
    

Sad
Sad
Sad
Sad
Sad
Sad
Sad
Sad
Sad
Sad
Sad
Sad
Sad
Sad
Sad
Sad
Sad
Sad
Sad
Sad
Sad
Sad
Sad
Sad
Sad
Sad
Sad
Sad
Sad
Sad
Sad
Sad
Sad
Sad
Sad
Sad
Sad


KeyboardInterrupt: 